In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
from functools import partial
from statistics import mean, stdev
import time


import utils
import generators
import bases
import representations
import eigensolvers
import const as C

def get_circle_obs_krylov(initial_state=0, num_col=200):
    return utils.generate_krylov(generators.angle_evolution, generators.full_state_observable, initial_state, num_col)

def get_zero_norm_ratio(V):
    N = V.shape[1]
    n = N
    for k in range(N):
        if np.linalg.norm(V[:,k]) < C.POWER_EPS:
            n -= 1
    return n/N

def run(U, basis=bases.fourier_basis, degree=10, representation=representations.EDMD_matrix_representation, n_run=10):
    K, V, L = representation(U, degree=degree, basis=basis)
    results = []
    iters = []
    for i in range(n_run):
        new_L, new_V, n = eigensolvers.power_orthogonal_QR_algorithm(K, partial(basis, degree=degree))
        r = get_zero_norm_ratio(new_V)*100
        results.append(r)
        iters.append(n)
        print("i={} new V: {:>5.2f}[%] iter {:>5}".format(i, r, n))
    print("ratio mean {:>5.1f}[%], std {:>5.2f}".format(mean(results), stdev(results)))
    print("iters mean {:>5.1f}, std {:>5.2f}".format(mean(iters), stdev(iters)))
    return results, iters

def pure_inv_run(U, basis=bases.fourier_basis, degree=10, representation=representations.EDMD_matrix_representation, n_run=10):
    K, V, L = representation(U, degree=degree, basis=basis)
    results = []
    iters = []
    for i in range(n_run):
        new_L, new_V, n = eigensolvers.QR_algorithm_with_inverse_iteration(K)
        r = get_zero_norm_ratio(new_V)*100
        results.append(r)
        iters.append(n)
        print("i={} new V: {:>5.2f}[%] iter {:>5}".format(i, r, n))
    print("ratio mean {:>5.1f}[%], std {:>5.2f}".format(mean(results), stdev(results)))
    print("iters mean {:>5.1f}, std {:>5.2f}".format(mean(iters), stdev(iters)))
    return results, iters

In [2]:
DEGREE = 100
N_RUN = 10

In [3]:
Ut = np.expand_dims(get_circle_obs_krylov(), axis=1)
print(f"Ut shape{Ut.shape}")

Ut shape(200, 1)


## params
POWER_EPS = 1e-1  
INV_EPS = 1e-9  
ARNOLDI_EPS = 1e-9  
DEGREE = 100  
N_RUN = 10  

In [4]:
ns_results, ns_iters = run(Ut, degree=DEGREE, n_run=N_RUN)

i=0 new V: 55.72[%] iter 128228
i=1 new V: 75.12[%] iter 169888
i=2 new V: 68.66[%] iter 156076
i=3 new V: 68.16[%] iter 154980
i=4 new V: 61.69[%] iter 141208
i=5 new V: 50.75[%] iter 117438
i=6 new V: 66.17[%] iter 150740
i=7 new V: 73.13[%] iter 165710
i=8 new V: 100.00[%] iter 221100
i=9 new V: 83.08[%] iter 186580
ratio mean  70.2[%], std 14.02
iters mean 159194.8, std 29559.86


In [5]:
print("ratio mean {:>5.2f}[%], std {:>5.2f}".format(mean(ns_results), stdev(ns_results)))
print("iters mean {:>5}, std {:>5.2f}".format(mean(ns_iters), stdev(ns_iters)))

ratio mean 70.25[%], std 14.02
iters mean 159194.8, std 29559.86


In [6]:
p_results, p_iters = pure_inv_run(Ut, degree=DEGREE, n_run=N_RUN)

i=0 new V: 100.00[%] iter 201000
i=1 new V: 100.00[%] iter 201000
i=2 new V: 100.00[%] iter 201000
i=3 new V: 100.00[%] iter 201000
i=4 new V: 100.00[%] iter 201000
i=5 new V: 100.00[%] iter 201000
i=6 new V: 100.00[%] iter 201000
i=7 new V: 100.00[%] iter 201000
i=8 new V: 100.00[%] iter 201000
i=9 new V: 100.00[%] iter 201000
ratio mean 100.0[%], std  0.00
iters mean 201000.0, std  0.00


In [7]:
mean(ns_results)/mean(p_results)

0.7024875621890547

In [8]:
100*99*98/2

485100.0

In [9]:
K, V, L = representations.EDMD_matrix_representation(Ut, basis=bases.fourier_basis, degree=100)

In [10]:
K.shape

(201, 201)